In [1]:
import stanza, re, benepar, psutil, gc, json, csv, time, string, os

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns

from textstat import textstat
from datetime import datetime
from collections import Counter

import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk import Tree
from textblob import Word
from spellchecker import SpellChecker

import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatter

In [ ]:
# These only need to run once
# stanza.download('en')
spacy.cli.download("en_core_web_sm")
nltk.download('punkt_tab')
benepar.download('benepar_en3')

In [5]:
source_texts= list()
source_texts.append("Kazuo Ishiguro - Never Let Me Go")
source_texts.append("Kazuo Ishiguro - The Remains of the Day")
source_texts.append("Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990)")
source_texts.append("Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_")
source_texts.append("The Buried Giant (Kazuo Ishiguro) (Z-Library)-1")
source_texts.append("Kazuo Ishiguro - The Unconsoled-Vintage (1996)")
directory_path = "C:/Users/Roland/Documents/AI/stylometry/"


Select choice of text by specifying index (starting with 0):

In [6]:
text_choice = 3

In [144]:
file_path = directory_path+source_texts[text_choice]+".txt"
file_path_edited = file_path[:-4]+" edited.txt"
file_path_edited_1 = file_path[:-4]+" edited_1.txt"
file_path_edited_2 = file_path[:-4]+" edited_2.txt"
print (file_path)
print(file_path_edited)
print(file_path_edited_1)
print(file_path_edited_2)

C:/Users/Roland/Documents/AI/stylometry/Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_.txt
C:/Users/Roland/Documents/AI/stylometry/Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_ edited.txt
C:/Users/Roland/Documents/AI/stylometry/Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_ edited_1.txt
C:/Users/Roland/Documents/AI/stylometry/Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_ edited_2.txt


Try reading the file with different encodings:

In [8]:
encodings = ['utf-8', 'ascii', 'latin-1','windows-1252', 'iso-8859-1', 'iso-8859-15']
texts = []
valid_encodings = list()
for i, encoding in enumerate(encodings):
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            test_text = file.read()
        print("""Successfully read the file with encoding: {encoding} (encoding index {ind}).  This gives the beginning as:\n{content}\n""".format(encoding=encoding, ind=i, content=test_text[0:200].replace("\n", "  ")))
        texts.append(test_text)
        valid_encodings.append(i)
    except UnicodeDecodeError:
        print(f"Failed to read with encoding: {encoding}")
print("valid encodings: ",valid_encodings)

Successfully read the file with encoding: utf-8 (encoding index 0).  This gives the beginning as:
    PART ONE    London, 24th July 1930    Chapter One      It was the summer of 1923, the summer I came down from Cambridge, when despite my aunt’s wishes that I return to Shropshire, I decided my future lay i

Failed to read with encoding: ascii
Successfully read the file with encoding: latin-1 (encoding index 2).  This gives the beginning as:
    PART ONE    London, 24th July 1930    Chapter One      It was the summer of 1923, the summer I came down from Cambridge, when despite my auntâs wishes that I return to Shropshire, I decided my future lay

Failed to read with encoding: windows-1252
Successfully read the file with encoding: iso-8859-1 (encoding index 4).  This gives the beginning as:
    PART ONE    London, 24th July 1930    Chapter One      It was the summer of 1923, the summer I came down from Cambridge, when despite my auntâs wishes that I return to Shropshire, I decided my

Are they the same?

In [9]:
for i in range(len(texts)):
    print (f"text {i} length {len(texts[i])}:-")
    for j, t in enumerate(texts):
        print (j, t == texts[i])

text 0 length 579045:-
0 True
1 False
2 False
3 False
text 1 length 590795:-
0 False
1 True
2 True
3 False
text 2 length 590795:-
0 False
1 True
2 True
3 False
text 3 length 590795:-
0 False
1 False
2 False
3 True


Now load text file with selected encoding

In [10]:
def check_for_file(file_path):
    if os.path.isfile(file_path):
        print("The file exists.")
    else:
        print("The file does not exist.")
    


In [12]:
check_for_file(file_path_edited)

The file does not exist.


In [147]:
for t in texts:
    with open(file_path_edited, 'w', encoding='utf-8') as file:
        file.write(t)
    with open(file_path_edited, 'r', encoding='utf-8') as file:
        text_read = file.read()
    print(text_read == t)

True
True
True
True


Watch out for this character as it is very awkward;

In [ ]:
print("first print")
print(ord("\x8e"))
print("second print")
print(chr(142))

Identify invalid characters using string:  Don't use this one as the later one is better.

In [ ]:

# Define a string containing only valid characters (alphanumeric + punctuation)
valid_chars = string.ascii_letters + string.digits + string.punctuation + " " = "\n"

# Example text with some invalid characters
#test_text = "This is an example text with invalid characters like Ò, ë, etc."
test_text = text
# Step 1: Identify invalid characters
invalid_chars = set([char for char in test_text if char not in valid_chars])

# Display the invalid characters found
print(f"Invalid characters found: {invalid_chars}")

# Step 2: Create a dictionary or list of tuples for replacement
# For simplicity, let's replace invalid characters with '?'
replacement_dict = {char: '?' for char in invalid_chars}

# If you want to create a list of tuples instead:
replacement_list = [(char, '?') for char in invalid_chars]

# Step 3: Replace invalid characters in the text
corrected_text = test_text
for char, replacement in replacement_dict.items():
    corrected_text = corrected_text.replace(char, replacement)

# Display the corrected text
#print(f"Corrected text: {corrected_text}")

# Output
print("Replacement dictionary:", replacement_dict)
print("Replacement list of tuples:", replacement_list)


Identify invalid characters using regular expressions (re):


In [14]:
# Define the pattern for valid characters (alphanumeric, spaces, and basic punctuation)
valid_char_pattern = re.compile(r'[a-zA-Z0-9\s.:;,?!\'"()£\-—…&]')

re_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

# Find invalid characters
    invalid_chars = set()
    for i, char in enumerate(test_text):
        if not valid_char_pattern.match(char):
            invalid_chars.add(char)

# Output the list of invalid characters and their positions
    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters using re for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    re_chars[enc] = invalid_chars

invalid characters using re for utf-8: {'*', '“', '‘', '”', '’'}
invalid characters using re for latin-1: {'\x9c', '\x9d', '*', '\x80', '\x98', '¦', '\x99', 'â'}
invalid characters using re for iso-8859-1: {'\x9c', '\x9d', '*', '\x80', '\x98', '¦', '\x99', 'â'}
invalid characters using re for iso-8859-15: {'\x9c', '\x9d', '*', '\x80', 'Š', '\x98', '\x99', 'â'}


In [15]:
# Define valid characters using the string module
valid_chars = string.ascii_letters + string.digits + string.punctuation + "…—àêé \n\t"
valid_chars = ''.join([char for char in valid_chars if char not in "{}~$%"])
print (f"valid characters used by string: {string.ascii_letters + string.digits + ' ' + string.punctuation}")

string_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

    invalid_chars = set()
    for i, char in enumerate(test_text):
        if char not in valid_chars:
            invalid_chars.add(char)
    #invalid_chars.difference_update({"\n","\t"})

    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters found using string for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    string_chars[enc] = invalid_chars



valid characters used by string: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
invalid characters found using string for utf-8: {'“', '”', '‘', '’'}
invalid characters found using string for latin-1: {'\x9c', '\x9d', '\x80', '\x98', '¦', '\x99', 'â'}
invalid characters found using string for iso-8859-1: {'\x9c', '\x9d', '\x80', '\x98', '¦', '\x99', 'â'}
invalid characters found using string for iso-8859-15: {'\x9c', '\x9d', '\x80', 'Š', '\x98', '\x99', 'â'}


In [17]:
for char in invalid_chars:
    print(ord(char)," ", char)


157   
160    
226   â
195   Ã
37   %
148   
153   
12   
152   
128   
167   §
156   
162   ¢
352   Š


In [ ]:
#just a little test:
''.join([char for char in "abcdefgh" if char not in "cd"])

In [ ]:
print(valid_chars)
print(ord(valid_chars[68]))  #this is the ' after the & in the list
print(ord(valid_chars[-13])) #this is the ` before the { in the list
print(ord("'"))
print(ord('"'))
print(ord("`"))

In [16]:
def display_chars(chars, method, unprintable = 142):
    number = len(chars)
    invalid_chars = '  '.join(char for char in chars)
    unprintable_message = " + unprintable character" if chr(unprintable) in invalid_chars else ""
    print(f"{number} invalid chars found using {method}:  {invalid_chars}{unprintable_message}")

In [17]:
for enc, encoding_index in enumerate(valid_encodings):
    print(f"with encoding {encodings[encoding_index]}")
    
    display_chars(re_chars[enc], "re   ")
    display_chars(string_chars[enc], "string")
        
    re_only = re_chars[enc] - string_chars[enc]
    string_only = string_chars[enc] - re_chars[enc]
    
    print(f"\n{len(re_only)} chars in re but not in string:    {'  '.join(char for char in re_only)}")
    print(f"{len(string_only)} chars in string but not in re:    {'  '.join(char for char in string_only)}\n")

with encoding utf-8
5 invalid chars found using re   :  *  “  ‘  ”  ’
4 invalid chars found using string:  “  ”  ‘  ’

1 chars in re but not in string:    *
0 chars in string but not in re:    

with encoding latin-1
8 invalid chars found using re   :      *      ¦    â
7 invalid chars found using string:          ¦    â

1 chars in re but not in string:    *
0 chars in string but not in re:    

with encoding iso-8859-1
8 invalid chars found using re   :      *      ¦    â
7 invalid chars found using string:          ¦    â

1 chars in re but not in string:    *
0 chars in string but not in re:    

with encoding iso-8859-15
8 invalid chars found using re   :      *    Š      â
7 invalid chars found using string:        Š      â

1 chars in re but not in string:    *
0 chars in string but not in re:    



In [ ]:
print(ord("é"))
a = "eé".encode('utf-8')
print(len(a))

In [18]:
def compare_texts(text1, text2):
    if len(text1) != len(text2):
        #raise ValueError("Strings must be of the same length!")
        
# Create a list of tuples with differing characters
    differences = [(char1, char2) for char1, char2 in zip(text1, text2) if char1 != char2]

# Print the list of tuples
    print(differences[:100])



In [154]:
def find_first_difference(text_1, text_2):
    i = 0
    max_length = min(len(text_1),len(text_2))
    while ((text_1[:i] == text_2[:i]) and (i < max_length)):
        i = i + 1
    if i < max_length:
        print (f"first text, position {i} is {text_1[i]}")
        print(text_1[i:i+20])
        print (f"second text, position {i} is {text_2[i]}")
        print(text_2[i:i+20])
    else:
        print("no differences up to end of the smaller text")

In [155]:
# compare_texts(texts[0], texts[1])
find_first_difference(texts[0],texts[1])

first text, position 137 is s
s wishes that I retu
second text, position 137 is 
s wishes that I re


Now we are going to display all the invalid characters in their context to decide what to do about them. We will just do it for one of the versions as they are all the same.

In [20]:
def display_invalid_chars(test_text, window_size = 200, max_to_display = 8):
# Find all invalid characters and their positions
    #valid_chars = string.ascii_letters + string.digits + string.punctuation + " —…()£-&\n\t"
    #print(valid_chars)
    invalid_char_positions = [(i, char) for i, char in enumerate(test_text) if char not in valid_chars]
                          
    icp_list_sorted = sorted(invalid_char_positions,key=lambda x: x[1])
    
    for pos, char in icp_list_sorted:
        start = max(pos - window_size // 2, 0)
        end = min(pos + window_size // 2, len(test_text))
        window = test_text[start:end]
    
    # Highlight the invalid character in the window
        highlighted_window = window[:pos-start] + f"[{char}]" + window[pos-start+1:]
        highlighted_window = highlighted_window.replace("\n", " ")
        highlighted_window = highlighted_window.replace("\t", " ")
        print(f"Invalid character '{char}' ({ord(char)}) at position {pos}:\n{highlighted_window}\n")




In [42]:
def display_text_in_context(source_text,target_text,window = 50, repeat = False):
    ind = source_text.find(target_text)
    if not repeat:
        if ind != -1:
            window = source_text[max(0, ind - window): ind + window + len(target_text)]
            window = window.replace("\n", " ")
            window = window.replace("\t", " ")
            print(window)
        print("not found")
    else:
        if ind == -1:
            print("not found")
        while True:
            ind = source_text.find(target_text, ind)
            if ind == -1:
                break
            start_window = max(0, ind - window)
            end_window = min(len(source_text), ind + len(target_text) + window)
            window_text = source_text[start_window:end_window]
            print(ind, window_text)
            ind += 1  # Move to the next position
        
        
        
        
#        while ind != -1:
#            display_window = source_text[max(0, ind - window): ind + window + len(target_text)]
#            display_window = display_window.replace("\n", " ")
#            display_window = display_window.replace("\t", " ")
#            print(display_window)
#            source_text = source_text[ind:]


In [139]:
display_invalid_chars(texts[0][0:5000])

Invalid character '‘' (8216) at position 2683:
e rose and began his pacing again, pronouncing as he did so upon his various plans for the future.  [‘]I’ve a mind to go into publishing, you know. Newspapers, magazines, that sort of thing. In fact, I 

Invalid character '‘' (8216) at position 3031:
ally have no idea what you want to do? Look, it’s all out there for us’ - he indicated the window - [‘]Surely you have some plans.’  ‘I suppose so,’ I said, smiling. ‘I have one or two things in mind. I

Invalid character '‘' (8216) at position 3062:
 to do? Look, it’s all out there for us’ - he indicated the window - ‘Surely you have some plans.’  [‘]I suppose so,’ I said, smiling. ‘I have one or two things in mind. I’ll let you know in good time.’

Invalid character '‘' (8216) at position 3095:
for us’ - he indicated the window - ‘Surely you have some plans.’  ‘I suppose so,’ I said, smiling. [‘]I have one or two things in mind. I’ll let you know in good time.’  ‘What have you got up your sl

In [50]:
display_text_in_context(texts[0], "*", window = 60, repeat = True)

45959 grasped his hand still sitting. Then I remember him saying: *My poor lad. First your father. Now your mother. Must feel l


In [137]:
display_invalid_chars(texts[1][0:5000])

Invalid character '' (128) at position 137:
hapter One   It was the summer of 1923, the summer I came down from Cambridge, when despite my auntâ[]s wishes that I return to Shropshire, I decided my future lay in the capital and took up a small f

Invalid character '' (128) at position 2375:
at of old schoolfriends. I remember we spent a little time discussing the activities of the workersâ[] unions, before embarking on a long and enjoyable debate on German philosophy, which enabled us to

Invalid character '' (128) at position 2688:
 rose and began his pacing again, pronouncing as he did so upon his various plans for the future.  â[]Iâve a mind to go into publishing, you know. Newspapers, magazines, that sort of thing. In fact,

Invalid character '' (128) at position 2692:
e and began his pacing again, pronouncing as he did so upon his various plans for the future.  âIâ[]ve a mind to go into publishing, you know. Newspapers, magazines, that sort of thing. In fact, I f

I

In [32]:
display_invalid_chars(texts[2])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [33]:
display_invalid_chars(texts[3])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
print("€" in texts[3])
print(texts[3][246245])

In [24]:
# Function to find and replace specified strings in a text
def replace_strings_with_context(text, replacements, window=10, silent = False):
    """
    Replaces specified strings within a text and prints the context around each replacement.
    
    Parameters:
    - text: The original string where replacements are done.
    - replacements: A dictionary where keys are the substrings to find, and values are their replacements.
    - window: Number of characters to show before and after the replaced string for context.
    
    Returns:
    - A modified string with the replacements applied.
    """
    counter = 0
    for old, new in replacements.items():
        index = text.find(old)
        while index != -1:
            # Extract context around the found string
            start = max(index - window, 0)
            end = min(index + len(old) + window, len(text))
            before = text[start:index]
            after = text[index + len(old):end]
            before = before.replace("\n", " ")
            before = before.replace("\t", " ")
            after = after.replace("\n", " ")
            after = after.replace("\t", " ")

            # Print before, replaced, and after strings
            if not silent:
                print(f"Before: {before}[{old}]{after}")
                print(f"After:  {before}[{new}]{after}\n")

            # Replace the string in the text
            text = text[:index] + new + text[index + len(old):]
            counter += 1
            
            # Find the next occurrence of the old string
            index = text.find(old, index + len(new))

    print(f"made {counter} changes")
    return text




In [ ]:
print(ord("b"))
print(chr(98))
print(ord("Ž"))

In [100]:
replacements = dict()
# replacements["t}:le"] = "the"
# replacements["up o~ the"] = "up on the"
# replacements["the ~ort"] = "the sort"
# replacements["than ~ might"] = "than might"
# replacements["the house~"] = "the house."
# replacements["while ~earlier"] = "while earlier"
# replacements["meetings ~once"] = "meetings once"
# replacements["Ž"] = "é"
# replacements["mean- it"] = "mean it"
# replacements["¤"] = "s"
# replacements["Ð"] = "-"
# replacements["É"] = '-'
# replacements["Ó"] = '"'
# replacements["Ò"] = '"'
# replacements["Ô"] = '"'
# replacements["Rover.Ó I thanked"] = "Rover  I thanked"
# replacements['Rover." I thanked'] = "Rover  I thanked"
# replacements["Õ"] = "'"
# replacements[chr(136)] = "à"
# replacements[chr(142)] = "é"
# replacements[chr(144)] = "ê"
# replacements["Ñ"] = "-"
# replacements[chr(12)] = ""
# replacements["%egan"] = "began"
# replacements["t4arijço"]="Sachiko"
replacements[chr(8217)] = "'"
replacements[chr(8216)] = '"'
replacements[chr(8220)] = '"'
replacements[chr(8221)] = '"'
replacements["*"] = '"'
replacements["AH the same"] = "Ah the same"
# replacements["•"] = ""

for key, value in replacements.items():
        print(f"{key}: {value}")

’: '
‘: "
“: "
”: "
*: "
AH the same: Ah the same


In [101]:
text_edited = replace_strings_with_context(texts[0],replacements, silent = True)

made 5749 changes


Now see if there are any problems left:

In [156]:
display_invalid_chars(text_edited)


In [157]:
display_text_in_context(text_edited, "Chapter", window = 40, repeat = True)
display_text_in_context(text_edited, "PART", window = 80, repeat = True)

36 

PART ONE

London, 24th July 1930

Chapter One


It was the summer of 1923, the su
38821 lous preoccupations to deflect me.





Chapter Two


Since I am now recalling that per
65599 mmings. Now, if you'll excuse me.'





Chapter Three


It took no more than a few days
91099 

PART TWO


London, 15th May 1931





Chapter Four


At the rear of our garden in Sha
127779  that argument in the dining room.





Chapter Five


I do not remember now if the din
145625 d of the course things would take.





Chapter Six


It was around that same time - th
165071  being unloaded off the motor car.





Chapter Seven


Akira, I was delighted to learn
199639 lways be proud of what he's done.'





Chapter Eight


I do not remember much about th
225770 d, I intend before long to embark.





Chapter Nine


There is a further aspect to thi
238757 RT THREE


London, 12th April 1937





Chapter Ten


Yesterday, by the time young Jenn
261910 emmings at the Draycoats' wedding.





Chapter Eleven

In [104]:
print(ord("\n"))

10


In [70]:
titles = list()
titles.append(("PART ONE", "Chapter One"))
titles.append(("PART TWO", "Chapter Four"))
titles.append(("PART THREE", "Chapter Ten"))
titles.append(("PART FOUR", "Chapter Twelve"))
titles.append(("PART FIVE", "Chapter Fourteen"))
titles.append(("PART SIX", "Chapter Sixteen"))
titles.append(("PART SEVEN", "Chapter Twenty-three"))




for edit_start, edit_end in titles:
    print(edit_start, edit_end)


PART ONE Chapter One
PART TWO Chapter Four
PART THREE Chapter Ten
PART FOUR Chapter Twelve
PART FIVE Chapter Fourteen
PART SIX Chapter Sixteen
PART SEVEN Chapter Twenty-three


In [89]:
def remove_chunks(text, chunks, remove_all = False, buffer = 100):
    # removes chunks defines by a list of tuples, start of chunk to end of chunk, removes the second part only if remove_all is set to True
    for edit_start, edit_end in chunks: 
        ind1 = text.find(edit_start)
        ind2 = text.find(edit_end)
        if remove_all:
            ind2 += len(edit_end)
        if ind2 > ind1 and ind2 < ind1 + 100 and ind1 != -1 and ind2 != -1:
            print (f"removing {edit_start} to {edit_end}")
            text = text[:ind1] + text[ind2:]
        else:
            print(f"{edit_start} to {edit_end} not removed")
    return text


In [90]:
#TEST
tt = "The quick brown fox jumps over the lazy dog"
tt1 = [("quick", "the ")]
tt2 = remove_chunks(tt, tt1, remove_all = True)
print(tt)
print(tt2)

removing quick to the 
The quick brown fox jumps over the lazy dog
The lazy dog


In [158]:
text_edited_1 = remove_chunks(text_edited, titles, remove_all = True)
    


removing PART ONE to Chapter One
removing PART TWO to Chapter Four
removing PART THREE to Chapter Ten
removing PART FOUR to Chapter Twelve
removing PART FIVE to Chapter Fourteen
removing PART SIX to Chapter Sixteen
removing PART SEVEN to Chapter Twenty-three


In [167]:
tidy_up = dict()
tidy_up["  "] = " " 
tidy_up["Chapter Seventeen"] = ""
tidy_up["Chapter Eighteen"] = ""
tidy_up["Chapter Nineteen"] = ""
tidy_up["Chapter Two"] = ""
tidy_up["Chapter Three"] = ""
tidy_up["Chapter Five"] = ""
tidy_up["Chapter Six"] = ""
tidy_up["Chapter Seven"] = ""
tidy_up["Chapter Eight"] = ""
tidy_up["Chapter Nine"] = ""
tidy_up["Chapter Eleven"] = ""
tidy_up["Chapter Thirteen"] = ""
tidy_up["Chapter Fifteen"] = ""
tidy_up["Chapter Twenty-one"] = ""
tidy_up["Chapter Twenty-two"] = ""
tidy_up["Chapter Twenty"] = ""
tidy_up["The End"] = ""


In [168]:
text_edited_2 = replace_strings_with_context(text_edited_1,tidy_up, silent = False)

Before: go.'      [Chapter Seventeen]   I went 
After:  go.'      []   I went 

Before: car.      [Chapter Eighteen]   The pol
After:  car.      []   The pol

Before: him.      [Chapter Nineteen]   It is o
After:  him.      []   It is o

Before:  me.      [Chapter Two]   Since I
After:   me.      []   Since I

Before: me.'      [Chapter Three]   It took
After:  me.'      []   It took

Before: oom.      [Chapter Five]   I do no
After:  oom.      []   I do no

Before: ake.      [Chapter Six]   It was 
After:  ake.      []   It was 

Before: car.      [Chapter Seven]   Akira, 
After:  car.      []   Akira, 

Before: ne.'      [Chapter Eight]   I do no
After:  ne.'      []   I do no

Before: ark.      [Chapter Nine]   There i
After:  ark.      []   There i

Before: ing.      [Chapter Eleven]   It is n
After:  ing.      []   It is n

Before: urn.      [Chapter Thirteen]   I spent
After:  urn.      []   I spent

Before: se.'      [Chapter Fifteen]   For the
After:  se.'      []   For the

Be

In [160]:
display_text_in_context(text_edited_2, "Chapter")

not found


In [97]:
def find_capitalized_words(text):
    """
    Finds all words in the string that are in capitals, excluding words of one character.
    
    Parameters:
    - text: The input string.
    
    Returns:
    - A list of fully capitalized words with more than one character.
    """
    # Use regex to match words with all capital letters and exclude single-character words
    capitalized_words = re.findall(r'\b[A-Z]{2,}\b', text)
    return capitalized_words


In [99]:
capitalized_words = find_capitalized_words(text_edited_2)
for word in capitalized_words:
    if word != "OK":
        ind = text_edited.find(word)
        window = text_edited[ind-75:ind+40]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)


e compensation money. Miss Givens said we could go shopping on Tuesday.'  "AH the same. Look, I think you're awfull
id. "After all, you were one of the finest detectives ever to serve in the SMP.'  "Really? How kind of you to say s


In [169]:
print(file_path_edited_2)

C:/Users/Roland/Documents/AI/stylometry/Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_ edited_2.txt


In [171]:
print(len(text_edited_2))

578362


In [172]:
with open(file_path_edited_2, 'w', encoding='utf-8') as f:
    f.write(text_edited_2)

Check that saving and loading the corrected file doesn't change it.  There will be a change when it encounters é because it takes 2 bytes.  Up to that point, there should be no differences.

In [173]:
with open(file_path_edited_2, 'r', encoding='utf-8') as f:
    text = f.read()

In [174]:
text == text_edited_2

True

In [175]:
text_edited.find("é")

-1

In [113]:
print(len(text), len(text_edited_2))

578625 578369


In [132]:
i = 0
max_length = min(len(text),len(text_edited_2))
while ((text[:i] == text_edited_2[:i]) and (i < max_length)):
    i = i + 1
if i < max_length:
    print(text[i:i+100])
    print(text_edited_2[i:i+100])
else:
    print("no differences up to end of the smaller text")

€¦'

"Quite all right. Perfectly natural, after all. Young fellow like you.'

"I assure you, sirâ€¦'
'

"Quite all right. Perfectly natural, after all. Young fellow like you.'

"I assure you, sir…'

"N


In [133]:
for i in range(2):
    print(text[i*1000:i*1000+1000])
    print("**************************************")
    print(text_edited_2[i*1000:i*1000+1000])
    print("*********************************************************************************")
    print("*********************************************************************************")
    






It was the summer of 1923, the summer I came down from Cambridge, when despite my aunt's wishes that I return to Shropshire, I decided my future lay in the capital and took up a small flat at Number 14b Bedford Gardens in Kensington. I remember it now as the most wonderful of summers. After years of being surrounded by fellows, both at school and at Cambridge, I took great pleasure in my own company. I enjoyed the London parks, the quiet of the Reading Room at the British Museum; I indulged entire afternoons strolling the streets of Kensington, outlining to myself plans for my future, pausing once in a while to admire how here in England, even in the midst of such a great city, creepers and ivy are to be found clinging to the fronts of fine houses.

It was on one such leisurely walk that I encountered quite by chance an old schoolfriend, James Osbourne, and discovering him to be a neighbour, suggested he call on me when he was next passing. Although at that point I had yet to rece

In [115]:
text_edited_2.find(chr(8216))

-1

In [120]:
text_edited_1.find(chr(8230))

19068

In [122]:
print(text_edited_1[200:230])
print(text_edited_1[19000:19120])
      

nese name, and I— perhaps out 
f the
city.'
"Oh really? I didn't realize that. You lived there with…with friends?"
Sachiko stopped pouring and looked a


In [164]:

# Sample text with hyphenated words across lines
t = texts[0]
# Use regex to find words that are hyphenated across lines
# Match sequences where a hyphen is at the end of a line, followed by a newline, and then continued with a word
hyphenated_words = re.findall(r"(\w+)-\n(\w+)", t)

# Print the hyphenated words
for first_part, second_part in hyphenated_words:
    print(f"Found broken word: {first_part}-{second_part}")
print(len(hyphenated_words))
print(len(t))


0
579045


In [166]:
print(text_edited_2[-2000:])

ef that for you, it was the correct decision not to come with me that day. You always felt you had a mission to complete, and I dare say you would never have been able to give your heart to anyone or anything until you had done so. I can only hope that by now your tasks are behind you, and that you too have been able to find the sort of happiness and companionship which I have come lately almost to take for granted.'

There is something about these sections of her letter - and those last lines in particular - that never quite ring true. Some subtle note that runs throughout the letter - indeed, her very act of writing to me at that moment - feels at odds with her report of days filled with "happiness and companionship'. Was her life with her French count really what she set off to find that day she stepped out on to the jetty in Shanghai? I somehow doubt it. My feeling is that she is thinking of herself as much as of me when she talks of a sense of mission, and the futility of attempti